# Stock Trading Environment
### Overview
A reinforcement learning environment for trading stock securities. This environment accepts pairs at which point it will automatically fetch all financial information.
### References
|Reference|Relevance|
|--|--|
|[OpenAI Gym Base](https://github.com/openai/gym/blob/master/gym/core.py)|The base class for our environment. This interface seems to be a standard in the reinforcement learning space.|

### Import Dependencies

In [5]:
from enum import Enum
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorforce 0.6.5 requires numpy==1.19.5, but you have numpy 1.22.4 which is incompatible.
tensorforce 0.6.5 requires tensorflow==2.6.0, but you have tensorflow 2.9.1 which is incompatible.
konoha 4.6.5 requires importlib-metadata<4.0.0,>=3.7.0, but you have importlib-metadata 6.0.0 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 721.7/721.7 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827641 sha256=4376931be90e85eb5f2d1a3c8c61804cfd08902fefe726e8d32af35c0765998f
  Stored in directory: c:\users\divanma\appdata\local\pip\cache\wheels\fc\bf\16\63ad354aa94e522b5fddf1729d8dfc47ad5980079054d4f714
Successfully built gym
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.1
    Uninstalling importlib-metadata-3.10.1:
      Successfully uninstalled importlib-metadata-3.10.1
  Attempting uninstall: gym
    Found existing installation: gym 0.21.0
    Uninstallin

### Action Space
This refers to the decisions / actions that can be applied to the environment. Usually decided by some intelligent system like a neural network or a state vector machine.

In [2]:
class Actions(Enum):
    Hold = 0 # Perform no action.
    Buy = 1 # Buy a new asset.
    Sell = 2 # Sell an existing asset.

### Define the Environment

In [58]:
class StockTradingEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    config_supported_stock_codes = {}
    config_tax_income_ratio = 0.3
    config_tax_capital_gains_ratio = 0.16
    config_tax_capital_gains_min_years = 3
    state_transactions = None

    '''
    Observation Space   : [ [ <Code>, <Total shares>, <Total gain> ](3) ](Count of supported codes)
    Action Space        : [ <Count of supported codes / symbols.>, <Count of supported actions (Hold, Buy, Sell).>, <Max percentage of portfolio per-transaction.> ]
    '''
    def __init__(self, supported_stock_codes):
        if supported_stock_codes is None or len(supported_stock_codes) <= 0:
            raise Exception('One or more supported stock codes are required.')

        self.config_supported_stock_codes = { s:None for s in supported_stock_codes }
        self.action_space = self.__get_action_space__(len(supported_stock_codes), len(Actions))
        self.observation_space = self.__get_observation_space__(len(supported_stock_codes))

        self.reset()

    def __get_action_space__(self, nr_of_options: int, nr_of_actions: int, max_portfolio_ratio: int = 100) -> gym.Space:
        return spaces.MultiDiscrete([ nr_of_options, nr_of_actions, max_portfolio_ratio ])

    def __get_observation_space__(self, nr_of_options: int) -> gym.Space:
        return spaces.Box(low=0, high=np.iinfo(np.int16).max, dtype=np.int16, shape=(nr_of_options, 3))

    def __populate_financial_cache_and_get_available_codes__(self, codes_to_fetch):
        return codes_to_fetch

    def step(self, action):
        observation = None
        step_reward = None
        is_terminal = False
        additional_info = None

        print('Stepping in the environment.')

        return observation, step_reward, is_terminal, additional_info

    def reset(self):
        # Fetch financial data for supported codes and return which ones are still available.
        currently_available_supported_stock_codes = self.__populate_financial_cache_and_get_available_codes__([s for s in self.config_supported_stock_codes])
        
        print(f'Resetting the environment with {len(self.config_supported_stock_codes)} supported stock codes.')
        # Resetting transactions.
        self.state_transactions = pd.DataFrame(currently_available_supported_stock_codes, columns=['code']).set_index('code')
        self.state_transactions['shares'] = 0
        self.state_transactions['gain'] = 0

    def render(self, mode='human'):
        assert mode == 'human', 'Currently only human-mode is supported for rendering with rgb_array support coming.'

        print('Rendering the environment.')

codes = [ 'DDD', 'TSLA' ]
env = StockTradingEnv(codes)

env.state_transactions

Resetting the environment with 2 supported stock codes.


,shares,gain
code,,
DDD,0,0
TSLA,0,0
